In [ ]:
import numpy as np
import pandas as pd
import category_encoders as ce

In [ ]:
# Don't forget to change it for your computer. Use the 'clean' verion (from Ridha's Notebook)
path_to_clean_data_SA = '../../../ML_EBOLA/Cleaned_data/SA_clean/SA_clean.csv'
path_to_clean_data_DD = '../../../ML_EBOLA/Cleaned_data/DD_clean.csv'
path_to_clean_data_DM = '../../../ML_EBOLA/Cleaned_data/DM_clean.csv'

In [ ]:
df_SA = pd.read_csv('../../../ML_EBOLA/Cleaned_data/SA_clean/SA_clean.csv')

# Categorical data in SA
We replace the categorical features such as symptoms by each symptom having one column and then 0 or 1 for postiive


In [ ]:
df_SA.info()

In [ ]:
df_SA['SAMODIFY'].value_counts()

In [ ]:
sum(df_SA['SAMODIFY'].isna())

In [ ]:
# You need to import : import category_encoders as ce (pip install)
name = ce.OneHotEncoder(handle_unknown='ignore', use_cat_names=True)

# Then you fit the column which you wish to encode
symptoms = name.fit_transform(df_SA['SAMODIFY'])



In [ ]:
# Now carefull ! we have transformed also the NaN into an encoding value (see 8-to-last)
symptoms.info() 

In [ ]:
#This removes the 9 first letters of the header for each column (removes SAMODIFY_)
symptoms.columns = [ x[9:] for x in symptoms.columns ] 

In [ ]:
# This removes the column of NaN's (up for debate)
symptoms.drop(columns= 'nan', inplace= True)

In [ ]:
symptoms # See, happy

In [ ]:
# I am removing SAMODIFY AND SATERM 

"""
cols = [5,6]#[0,2]
df_SA.drop(df_SA.columns[cols], axis =1, inplace = True)
"""
#alternate writing
cols = ['SAMODIFY', 'SATERM']
df_SA.drop(columns= cols, inplace= True)

In [ ]:
df_SA

In [ ]:
# this is the file were we add the one hot encoding ! 
df_SA_symp = df_SA.join(symptoms)

In [ ]:
df_SA_symp

In [ ]:
# Now we have the symptoms 
# Let's import the result (EBOLA/OTHER) and also some demographics (age and sex)
df_DD = pd.read_csv(path_to_clean_data_DD)


In [ ]:
df_DM = pd.read_csv(path_to_clean_data_DM)

In [ ]:
# Cause of death (We can see this needs to be changed....)
# TWO categories, EBOLA vs OTHERS 
df_DD['DDORRES']

def parse_death_causes(cause):
    ebola = ['EBOLA', 'DIED EBOLA POSITIVE', 'PROBABLE EBOLA','EVD', 'EBOLA SURVIVOR DIED BEFORE DISCHARGE']
    if(cause in ebola):
        return "EBOLA"
    else:
        return "OTHER"

df_DD.loc[:,'DDORRES'] = df_DD['DDORRES'].apply(lambda x: parse_death_causes(x))
death_causes_count = df_DD['DDORRES'].value_counts()

In [ ]:
df_DD['DDORRES'].value_counts()

In [ ]:
df_DD['USUBJID'].value_counts()

In [ ]:
"""cols = [0,1,2,9,10]
df_DD.drop(df_DD.columns[cols], axis =1, inplace = True)
"""
# Alternate writing
colsDD = ['Unnamed: 0', 'STUDYID', 'DOMAIN', 'DDSTRTPT', 'DDSTTPT']
df_DD.drop(columns= colsDD, inplace= True)
# the data set has now, EBOLA OR OTHER :)
# Let me just drop many columns 


In [ ]:
"""colsDM = [0,1,2,6,10,11,12,14]
df_DM.drop(df_DM.columns[colsDM], axis =1, inplace = True)"""
colsDM = ['Unnamed: 0', 'STUDYID', 'DOMAIN', 'SITEID', 'ARMCD', 'ARM', 'COUNTRY', 'CITIES']
df_DM.drop(columns= colsDM, inplace= True)

In [ ]:
# Let's only look at day one careful not the the same reference day right ..... ?  
df_DD1 = df_DD[df_DD['DDDY']==1.0]
df_DM1 = df_DM[df_DM['DMDY'] ==1.0]

In [ ]:
# Let's merge 
Merged = df_DD1.merge(df_DM1, how = 'inner', on = 'USUBJID')

In [ ]:
Merged["DDORRES"].value_counts()

In [ ]:
Merged['SEX'] = Merged['SEX'].apply(lambda x: 1 if x == 'M' else 0)

In [ ]:
#This variable is a sequence number to ensure uniqueness of subject records within the domain. Each observation 
#(each recorded as a separate row in the table) will have a unique number within each subject.
Merged = Merged[Merged["DDSEQ"] == 1]

In [ ]:
# Questionnable choice... (But I'm in no position to say better)

In [ ]:
Merged

In [ ]:
Merged.USUBJID.nunique()

In [ ]:
df_Temp = Merged.copy()
df_Temp2 = df_Temp[df_Temp['DDORRES'] == "EBOLA"]
df_Temp2.USUBJID.nunique()

In [ ]:
df_SA_symp

In [ ]:
# Merge with symptoms 
df_ml = Merged.merge(df_SA_symp, how = 'inner', on = 'USUBJID')

In [ ]:
uniqueValues = df_ml['USUBJID'].nunique()
print("We have ",uniqueValues, " unique patients")
df_ml["DDORRES"].value_counts()

In [ ]:
# The same person is tagged with ebola mutliple times because we have different symptoms ? 

In [ ]:
df_ml.info()

In [ ]:
df_ml['SASCAT']

In [ ]:
"""# Drop non_null values 
df_ml.drop(df_ml.columns[[23,27]], inplace = True, axis = 1)"""

In [ ]:
# Very bruteforce hardfix to drop all NaN...
df_ml = df_ml.dropna(axis= 'columns')
df_ml.info()

In [ ]:
len(df_ml[df_ml.duplicated()])

In [ ]:
df_ml["DDSEQ"].value_counts()

In [ ]:
df_ml.info()

In [ ]:
# All age are in years you can drop AGEUnit
df_ml["AGEU"].value_counts()

# Now I'm bout to drop MANY MANY features (carefull I only kept DDSEQ ==1 for now )


In [ ]:
"""cols_ml = [
    0, # USUBJID : no need for the ID anymore
    1, # DDSEQ
    2, # DDTEST
    3, # DDTESTCD
    5, # DDDY
    6, # RFSTDTC
    7, # DTHFL
    9, # AGEU
    11,# DMDY
    12,# Unnamed: 0
    13,# STUDYID
    14,# DOMAIN
    15,# SASEQ
    16,# SACAT
    17,# SASCAT
    18,# SAPRESP
    19,# SAOCCUR  
    20,# SASTAT
    21,# SAREASND
    22,# SASEV
    23,# EPOCH 
    24,# SADY
    25 # SASTDY
]
df_ML = df_ml.drop(df_ml.columns[cols_ml], axis = 1)"""

# Alternative writing:
cols = [
    'USUBJID',
    'DDSEQ',
    'DDTEST',
    'DDTESTCD',
    'DDDY',
    'RFSTDTC',
    'DTHFL',
    'AGEU',
    'DMDY',
    'Unnamed: 0',
    'STUDYID',
    'DOMAIN',
    'SASEQ',
    #'SACAT',
    #'SASCAT',
    #'SAPRESP',
    #'SAOCCUR',
    #'SASTAT',
    #'SAREASND',
    #'SASEV',
    #'EPOCH',
    #'SADY',
    #'SASTDY'
]
df_ML = df_ml.drop(columns= cols, inplace= True)

In [ ]:
# Ok so here we are left with EBOLA, other, the symptoms, age and sex 
df_ml.info()

In [ ]:
# Try to see which columns are redundant...
worry_columns = []
for x in df_ml.columns:
    if df_ml[x].value_counts().size == 1:  # i.e. if there is only one value in the whole column, it is strange
        worry_columns.append(x)
        
worry_columns

In [ ]:
### !!! Need to see with someone who knows more...

In [ ]:
# Now drop those ~meaningless columns 
df_ml.drop(columns= worry_columns, inplace= True)

df_ml

In [ ]:
# Let's try and model
from sklearn.preprocessing import StandardScaler


#select the features needed
df_triage = df_ml

#transform the dependent variable
dict_fstatus = {'EBOLA' : 1, 'OTHER' : 0}
df_triage['DDORRES'] = df_triage['DDORRES'].apply(lambda x : dict_fstatus[x]) 

In [ ]:
# df_triage['USUBJID'].value_counts()

In [ ]:
df_temp = df_triage.copy()
df_temp = df_temp[df_temp['DDORRES'] == 1]
df_temp['USUBJID'].unique()

In [ ]:
df_triage.apply(lambda x: sum(x.isna()),axis=0)
df_triage.dropna(how='any',inplace= True)

In [ ]:
#normalize the continuous variables 
scaler = StandardScaler()
df_triage[['AGE']] = scaler.fit_transform(df_triage[['AGE']])
df_triage.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(12, 8))
corr = df_triage.corr()
color = plt.get_cmap('coolwarm')
color.set_bad('lightgrey') 
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns ,annot=True, fmt=".2f", cmap = color)
plt.show()

In [ ]:

df_triage = df_triage.loc[:, (df_triage != 0).any(axis=0)]

In [ ]:
import statsmodels.api as sm

X = sm.add_constant(df_triage.loc[:, df_triage.columns != 'DDORRES'])
y = df_triage['DDORRES']

est_OLS = sm.OLS(y, X.astype(float)).fit()
print(est_OLS.summary())

In [ ]:
est_logit = sm.Logit(y, X.astype(float)).fit()
print(est_logit.summary())